<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Sentiment/Review_Sentiment_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import requests
import json

def sentiment(review):
  files = {
        'text': (None, '%s' % review),
                 #'["Come on,lets play together,Team performed well overall."]'),
        'api_key': (None, 'ZgdR6KHBIJprH1G1wBsiC0ENCqcsNNcAhhgMFw6SbG8'),
  }
  response = requests.post('https://apis.paralleldots.com/v4/sentiment_batch', files=files)

  f = open("response.json", "w")
  f.write(json.dumps(response.json(), indent=4))
  f.close()
  data = open("response.json", "r").read()
  data_parsed = json.loads(data)

  risp = data_parsed['sentiment'][0]

  massimo = max(risp['positive'],risp['negative'],risp['neutral'])

  if risp['negative'] == massimo:
    return 'negative' + ' (' + '%s' % risp['negative'] + ')'

  if risp['positive'] == massimo:
    return 'positive' + ' (' + '%s' % risp['positive'] + ')'

  if risp['neutral'] == massimo:
    return 'neutral' + ' (' + '%s' % risp['neutral'] + ')'

In [4]:
#AGGIORNO IL csv

#Da cambiare 
#nome_ristorante
#restaurant_ultimiUtenti_id = link condivisibile di (nomeRistorante)_ultimiUtenti.csv
#restaurant_sentiment = link condivisibile di (nomeRistorante)_Random_ReviewSentiment
#tot da cambiare: ci va l'ultimo numeretto nero presente nel csv (nomeRistorante)_Random_ReviewSentiment

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import pandas as pd
import csv
import time

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nome_ristorante = 'TupeloHoney'

# DOWNLOAD merge.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Sentiment > Random_Reviews > (nomeRistorante)_ultimiUtenti.csv)
#File dove ci sono id e recensione
restaurant_ultimiUtenti_id = '128opWrhFQnk6cQvlrIZQp8D3RevH4f8w'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': restaurant_update_id})
download.GetContentFile('%s' % nome_ristorante + '_ultimiUtenti.csv')

user_review = pd.read_csv('%s' % nome_ristorante + '_ultimiUtenti.csv')

#csv review analizzate per ristorante
# restaurant_sentiment = id del file csv con le review analizzate (si trova in data > Sentiment > Sentiment_Analysis > (nomeRistorante)_Random_ReviewSentiment.csv)
restaurant_sentiment = '1sIHc1BZcBWT3Iqd94-PaA4BF5elxcEYi'  # FILE ID, got on google drive with condivision link
download1 = drive.CreateFile({'id': restaurant_sentiment})
download1.GetContentFile('%s' % nome_ristorante + '_Random_ReviewSentiment.csv')

#creo e apro un file .csv per scrivere
file_parsed = open('%s' % nome_ristorante + '_Random_ReviewSentiment.csv', 'a')
    
#creo il writer
writer = csv.writer(file_parsed)

cnt = 1    #30 al minuto
api_number = 1    #max 300 reviews
tot = 1    #totale reviews

for i, rev in user_review.iterrows():
  #10% per ogni ristorante
  if tot >= 1394:
    if api_number <= 300:
      if cnt < 30 :
        id = rev['user_id']
        response = sentiment([rev['text_review']])
        writer.writerow(['%s' % id, '%s' % response])
        cnt = cnt + 1
        api_number = api_number + 1
      else:
        id = rev['user_id']
        response = sentiment([rev['text_review']])
        writer.writerow(['%s' % id, '%s' % response])
        api_number = api_number + 1
        time.sleep(61)
        cnt = 1
    else: 
      break
  else: 
    tot = tot + 1

#salvo e chiudo il file .csv
file_parsed.close()

download1.SetContentFile('%s' % nome_ristorante + '_Random_ReviewSentiment.csv') #The contents of the file
download1.Upload()

KeyboardInterrupt: ignored